## Imports

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # must be BEFORE torch/TF import
import warnings
warnings.filterwarnings("ignore")
import torch
import torch.optim as optim
import torch.nn.functional as F
from bytelatent.model.blt import ByteLatentTransformerArgs, ByteLatentTransformer
from chronos import MeanScaleUniformBins, ChronosConfig
from utils.train_utils import *
# from TLLM_data_provider.data_factory import data_provider
from data_provider.data_factory import data_provider
import argparse
from tqdm import tqdm
import time
torch.cuda.set_device(0)   # 0 here means “the first visible GPU”, i.e. physical #3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from bytelatent.tokenizers.constants import PAD_ID

## Args

In [3]:
## Training Args
vocab_size = 4096
batch_size = 64
seq_len = 96
learning_rate = 5e-4
weight_decay = 1e-2
epochs = 50
grad_accumulation_steps = 1
clip_grad = 1.0
seed = 42
warmup_steps = 0
min_lr_factor = 0.1
decay_lr = True
compile = True
output_dir = "output"
save_every = 5
compile = False
dataset_name = 'ETTh2'
features = 'S'

## Data Loader

In [4]:
config = {
    'task_name': 'pretrain_long_term_forecast',
    'dataset': dataset_name,
    'data' : dataset_name,
    'embed' : 'timeF',
    'root_path': 'dataset/ETT-small/',
    'data_path': f'{dataset_name}.csv',
    'features': features,
    'seq_len': seq_len,
    'label_len': seq_len - 1,
    'pred_len': 1
}

args = argparse.Namespace(
    batch_size=batch_size,
    freq='h' if 'h' in dataset_name else 'm',
    num_workers=2,
    subsample_pct=None,
    fix_seed=42,
    target='OT',
    shuffle=True,
    scale=True
)

train_dataset, train_loader = data_provider(args, config, flag='train')
validate_dataset, validate_loader = data_provider(args, config, flag='val')
# test_dataset, test_loader = data_provider(args, config, flag='test')
len(train_dataset), len(validate_dataset) 

train 8544
val 2880


(8544, 2880)

## Data Loader Demo

In [ ]:
for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(train_loader):
    batch_x = batch_x.to(device)
    batch_y = batch_y.to(device)
    batch_x_mark = batch_x_mark.to(device)
    batch_y_mark = batch_y_mark.to(device)
    x = batch_x.float().squeeze(-1)
    y = batch_y.float().squeeze(-1)
    break
x.shape, y.shape 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.gridspec import GridSpec

def plot_all_channels(x, sample_idx=0, channel_names=None, figsize=(15, 10)):
    """
    Plot all 7 channels for a single sample in the same plot
    
    Args:
        x: Input tensor/array of shape [batch_size, seq_len, n_channels]
        sample_idx: Which sample to plot (default: 0)
        channel_names: List of channel names (optional)
        figsize: Figure size tuple
    """
    
    # Extract single sample: [seq_len, n_channels]
    sample_data = x[sample_idx]  # Shape: [96, 7]
    seq_len, n_channels = sample_data.shape
    
    # Default channel names if not provided
    if channel_names is None:
        channel_names = [f'Channel {i+1}' for i in range(n_channels)]
    
    # Set up the plotting style
    plt.style.use('seaborn-v0_8' if 'seaborn-v0_8' in plt.style.available else 'default')
    colors = plt.cm.tab10(np.linspace(0, 1, n_channels))
    
    # Create figure with subplots
    fig = plt.figure(figsize=figsize)
    gs = GridSpec(3, 2, figure=fig, height_ratios=[2, 1, 1], hspace=0.3, wspace=0.3)
    
    # Main plot: All channels together
    ax_main = fig.add_subplot(gs[0, :])
    
    time_steps = np.arange(seq_len)
    
    for i in range(n_channels):
        channel_data = sample_data[:, i]
        ax_main.plot(time_steps, channel_data, 
                    color=colors[i], 
                    label=channel_names[i],
                    linewidth=1.5,
                    alpha=0.8)
    
    ax_main.set_title(f'All Channels - Sample {sample_idx}', fontsize=14, fontweight='bold')
    ax_main.set_xlabel('Time Steps')
    ax_main.set_ylabel('Values')
    ax_main.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax_main.grid(True, alpha=0.3)
    
    # Statistics subplot
    ax_stats = fig.add_subplot(gs[1, 0])
    
    # Calculate statistics for each channel
    stats_data = []
    for i in range(n_channels):
        channel_data = sample_data[:, i]
        stats_data.append([
            np.mean(channel_data),
            np.std(channel_data),
            np.min(channel_data),
            np.max(channel_data)
        ])
    
    stats_array = np.array(stats_data)
    stat_names = ['Mean', 'Std', 'Min', 'Max']
    
    # Heatmap of statistics
    im = ax_stats.imshow(stats_array.T, cmap='RdYlBu_r', aspect='auto')
    ax_stats.set_xticks(range(n_channels))
    ax_stats.set_xticklabels([f'Ch{i+1}' for i in range(n_channels)])
    ax_stats.set_yticks(range(len(stat_names)))
    ax_stats.set_yticklabels(stat_names)
    ax_stats.set_title('Channel Statistics')
    
    # Add text annotations
    for i in range(n_channels):
        for j in range(len(stat_names)):
            text = ax_stats.text(i, j, f'{stats_array[i, j]:.2f}',
                               ha="center", va="center", color="black", fontsize=8)
    
    # Correlation heatmap
    ax_corr = fig.add_subplot(gs[1, 1])
    
    # Calculate correlation matrix
    corr_matrix = np.corrcoef(sample_data.T)
    
    im_corr = ax_corr.imshow(corr_matrix, cmap='RdBu_r', vmin=-1, vmax=1, aspect='auto')
    ax_corr.set_xticks(range(n_channels))
    ax_corr.set_xticklabels([f'Ch{i+1}' for i in range(n_channels)])
    ax_corr.set_yticks(range(n_channels))
    ax_corr.set_yticklabels([f'Ch{i+1}' for i in range(n_channels)])
    ax_corr.set_title('Channel Correlations')
    
    # Add correlation values
    for i in range(n_channels):
        for j in range(n_channels):
            text = ax_corr.text(j, i, f'{corr_matrix[i, j]:.2f}',
                               ha="center", va="center", 
                               color="white" if abs(corr_matrix[i, j]) > 0.5 else "black", 
                               fontsize=8)
    
    # Individual channel plots (smaller)
    ax_individual = fig.add_subplot(gs[2, :])
    
    # Create small multiples for each channel
    for i in range(n_channels):
        offset = i * (np.max(sample_data) - np.min(sample_data)) * 1.2
        channel_data = sample_data[:, i] + offset
        ax_individual.plot(time_steps, channel_data, 
                          color=colors[i], 
                          label=channel_names[i],
                          linewidth=1)
        
        # Add channel label
        ax_individual.text(-2, offset + np.mean(sample_data[:, i]), 
                          f'Ch{i+1}', 
                          va='center', ha='right', 
                          color=colors[i], fontweight='bold')
    
    ax_individual.set_title('Individual Channels (Stacked View)')
    ax_individual.set_xlabel('Time Steps')
    ax_individual.set_ylabel('Values (with offset)')
    ax_individual.grid(True, alpha=0.3)
    ax_individual.set_yticks([])  # Remove y-ticks since they're offset
    
    plt.tight_layout()
    return fig

def plot_channel_distributions(x, sample_idx=0, channel_names=None, figsize=(12, 8)):
    """
    Plot distribution analysis for all channels
    """
    sample_data = x[sample_idx]
    seq_len, n_channels = sample_data.shape
    
    if channel_names is None:
        channel_names = [f'Channel {i+1}' for i in range(n_channels)]
    
    fig, axes = plt.subplots(2, 4, figsize=figsize)
    axes = axes.flatten()
    
    colors = plt.cm.tab10(np.linspace(0, 1, n_channels))
    
    for i in range(n_channels):
        ax = axes[i]
        channel_data = sample_data[:, i]
        
        # Histogram
        ax.hist(channel_data, bins=20, alpha=0.7, color=colors[i], edgecolor='black')
        ax.set_title(f'{channel_names[i]}')
        ax.set_xlabel('Value')
        ax.set_ylabel('Frequency')
        
        # Add statistics text
        mean_val = np.mean(channel_data)
        std_val = np.std(channel_data)
        ax.axvline(mean_val, color='red', linestyle='--', alpha=0.8, label=f'Mean: {mean_val:.2f}')
        ax.axvline(mean_val + std_val, color='orange', linestyle=':', alpha=0.6, label=f'+1σ: {mean_val + std_val:.2f}')
        ax.axvline(mean_val - std_val, color='orange', linestyle=':', alpha=0.6, label=f'-1σ: {mean_val - std_val:.2f}')
        ax.legend(fontsize=8)
    
    # Remove empty subplot
    if n_channels < len(axes):
        fig.delaxes(axes[-1])
    
    plt.suptitle(f'Channel Value Distributions - Sample {sample_idx}', fontsize=14, fontweight='bold')
    plt.tight_layout()
    return fig

def analyze_channel_independence(x, sample_idx=0, channel_names=None):
    """
    Analyze channel independence for univariate assumption validation
    """
    sample_data = x[sample_idx]
    seq_len, n_channels = sample_data.shape
    
    if channel_names is None:
        channel_names = [f'Channel {i+1}' for i in range(n_channels)]
    
    print(f"Channel Independence Analysis for Sample {sample_idx}")
    print("=" * 50)
    
    # Correlation analysis
    corr_matrix = np.corrcoef(sample_data.T)
    
    print("\nCorrelation Matrix:")
    print("-" * 30)
    for i in range(n_channels):
        for j in range(n_channels):
            if i != j:
                corr_val = corr_matrix[i, j]
                print(f"{channel_names[i]} vs {channel_names[j]}: {corr_val:.3f}")
    
    # High correlation pairs
    high_corr_pairs = []
    threshold = 0.7
    for i in range(n_channels):
        for j in range(i+1, n_channels):
            if abs(corr_matrix[i, j]) > threshold:
                high_corr_pairs.append((i, j, corr_matrix[i, j]))
    
    print(f"\nHigh Correlation Pairs (|r| > {threshold}):")
    print("-" * 40)
    if high_corr_pairs:
        for i, j, corr_val in high_corr_pairs:
            print(f"{channel_names[i]} - {channel_names[j]}: {corr_val:.3f}")
    else:
        print("No high correlation pairs found.")
    
    # Statistics comparison
    print("\nChannel Statistics:")
    print("-" * 20)
    for i in range(n_channels):
        channel_data = sample_data[:, i]
        print(f"{channel_names[i]}:")
        print(f"  Mean: {np.mean(channel_data):.3f}")
        print(f"  Std:  {np.std(channel_data):.3f}")
        print(f"  Range: [{np.min(channel_data):.3f}, {np.max(channel_data):.3f}]")
        print()
    
    return corr_matrix, high_corr_pairs

# Example usage:
def main():
    """
    Example of how to use the plotting functions
    """
    # Simulated data - replace with your actual x[0, :, i] data
    np.random.seed(42)
    batch_size, seq_len, n_channels = 32, 96, 7
    
    # Create synthetic data that mimics real time series
    x = np.random.randn(batch_size, seq_len, n_channels)
    
    # Add some trend and seasonality to make it more realistic
    for i in range(n_channels):
        trend = np.linspace(0, i*0.5, seq_len)
        seasonal = np.sin(2 * np.pi * np.arange(seq_len) / (20 + i*5)) * (0.5 + i*0.1)
        x[:, :, i] += trend + seasonal
    
    # ETT dataset channel names (common for ETT datasets)
    ett_channel_names = ['HUFL', 'HULL', 'MUFL', 'MULL', 'LUFL', 'LULL', 'OT']
    
    # Plot all channels
    fig1 = plot_all_channels(x, sample_idx=0, channel_names=ett_channel_names)
    plt.show()
    
    # Plot distributions
    fig2 = plot_channel_distributions(x, sample_idx=0, channel_names=ett_channel_names)
    plt.show()
    
    # Analyze independence
    corr_matrix, high_corr_pairs = analyze_channel_independence(x, sample_idx=0, channel_names=ett_channel_names)

if __name__ == "__main__":
    main()

In [ ]:
len(train_dataset), len(validate_dataset), len(test_dataset)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def analyze_ett_splits(seq_len=96):
    """
    Analyze the difference between pretrain=True and pretrain=False splits
    for ETT minute dataset
    """
    
    # ETT minute dataset constants
    # 12 months = 12 * 30 * 24 * 4 = 34,560 time points (15-min intervals)
    # 4 months = 4 * 30 * 24 * 4 = 11,520 time points  
    # 8 months = 8 * 30 * 24 * 4 = 23,040 time points
    
    month_minutes = 30 * 24 * 4  # 30 days * 24 hours * 4 (15-min intervals)
    
    # Calculate boundaries
    twelve_months = 12 * month_minutes  # 34,560
    four_months = 4 * month_minutes     # 11,520  
    eight_months = 8 * month_minutes    # 23,040
    
    print("ETT Minute Dataset Split Analysis")
    print("=" * 50)
    print(f"Total data points per month: {month_minutes:,}")
    print(f"12 months: {twelve_months:,} points")
    print(f"4 months: {four_months:,} points") 
    print(f"8 months: {eight_months:,} points")
    print(f"Sequence length: {seq_len}")
    print()
    
    # PRETRAIN = True splits
    print("PRETRAIN = True (Self-supervised pretraining)")
    print("-" * 45)
    
    pretrain_border1s = [
        0,  # Train start
        twelve_months + four_months - seq_len,  # Val start  
        twelve_months + four_months - seq_len   # Test start (same as val)
    ]
    pretrain_border2s = [
        twelve_months + four_months,  # Train end (16 months)
        twelve_months + eight_months, # Val end (20 months) 
        twelve_months + eight_months  # Test end (20 months)
    ]
    
    print("Train: [0 : 16 months]")
    print(f"  Start: 0, End: {pretrain_border2s[0]:,}")
    print(f"  Duration: {pretrain_border2s[0]:,} points ({pretrain_border2s[0]/month_minutes:.1f} months)")
    
    print("Val: [16 months - seq_len : 20 months]") 
    print(f"  Start: {pretrain_border1s[1]:,}, End: {pretrain_border2s[1]:,}")
    print(f"  Duration: {pretrain_border2s[1] - pretrain_border1s[1]:,} points ({(pretrain_border2s[1] - pretrain_border1s[1])/month_minutes:.1f} months)")
    
    print("Test: [16 months - seq_len : 20 months] (same as val)")
    print(f"  Start: {pretrain_border1s[2]:,}, End: {pretrain_border2s[2]:,}")
    print(f"  Duration: {pretrain_border2s[2] - pretrain_border1s[2]:,} points ({(pretrain_border2s[2] - pretrain_border1s[2])/month_minutes:.1f} months)")
    print()
    
    # PRETRAIN = False splits  
    print("PRETRAIN = False (Standard supervised learning)")
    print("-" * 48)
    
    normal_border1s = [
        0,  # Train start
        twelve_months - seq_len,  # Val start
        twelve_months + four_months - seq_len  # Test start
    ]
    normal_border2s = [
        twelve_months,  # Train end (12 months)
        twelve_months + four_months,  # Val end (16 months)
        twelve_months + eight_months  # Test end (20 months) 
    ]
    
    print("Train: [0 : 12 months]")
    print(f"  Start: 0, End: {normal_border2s[0]:,}")
    print(f"  Duration: {normal_border2s[0]:,} points ({normal_border2s[0]/month_minutes:.1f} months)")
    
    print("Val: [12 months - seq_len : 16 months]")
    print(f"  Start: {normal_border1s[1]:,}, End: {normal_border2s[1]:,}")
    print(f"  Duration: {normal_border2s[1] - normal_border1s[1]:,} points ({(normal_border2s[1] - normal_border1s[1])/month_minutes:.1f} months)")
    
    print("Test: [16 months - seq_len : 20 months]")
    print(f"  Start: {normal_border1s[2]:,}, End: {normal_border2s[2]:,}")
    print(f"  Duration: {normal_border2s[2] - normal_border1s[2]:,} points ({(normal_border2s[2] - normal_border1s[2])/month_minutes:.1f} months)")
    print()
    
    # Key differences
    print("KEY DIFFERENCES:")
    print("-" * 16)
    train_diff = pretrain_border2s[0] - normal_border2s[0]
    print(f"• Training data: Pretrain uses {train_diff:,} MORE points ({train_diff/month_minutes:.1f} more months)")
    print(f"• Val/Test overlap: Pretrain has OVERLAPPING val/test sets")
    print(f"• Purpose: Pretrain optimized for self-supervised learning")
    print()
    
    # Visualization
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 8))
    
    # Timeline in months
    total_months = 20
    months = np.arange(total_months + 1)
    
    # Pretrain splits
    ax1.barh(0, 16, left=0, height=0.3, color='blue', alpha=0.7, label='Train (16 months)')
    ax1.barh(0, 4, left=16-seq_len/month_minutes, height=0.15, color='orange', alpha=0.7, label='Val (4+ months)')  
    ax1.barh(-0.15, 4, left=16-seq_len/month_minutes, height=0.15, color='red', alpha=0.7, label='Test (4+ months)')
    ax1.set_xlim(0, 20)
    ax1.set_ylim(-0.4, 0.4)
    ax1.set_xlabel('Months')
    ax1.set_title('PRETRAIN = True (Self-supervised pretraining)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Normal splits
    ax2.barh(0, 12, left=0, height=0.3, color='blue', alpha=0.7, label='Train (12 months)')
    ax2.barh(0, 4, left=12-seq_len/month_minutes, height=0.15, color='orange', alpha=0.7, label='Val (4+ months)')
    ax2.barh(-0.15, 4, left=16-seq_len/month_minutes, height=0.15, color='red', alpha=0.7, label='Test (4+ months)')
    ax2.set_xlim(0, 20)
    ax2.set_ylim(-0.4, 0.4) 
    ax2.set_xlabel('Months')
    ax2.set_title('PRETRAIN = False (Standard supervised learning)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return {
        'pretrain_splits': {
            'train': (pretrain_border1s[0], pretrain_border2s[0]),
            'val': (pretrain_border1s[1], pretrain_border2s[1]), 
            'test': (pretrain_border1s[2], pretrain_border2s[2])
        },
        'normal_splits': {
            'train': (normal_border1s[0], normal_border2s[0]),
            'val': (normal_border1s[1], normal_border2s[1]),
            'test': (normal_border1s[2], normal_border2s[2])
        }
    }

def why_pretrain_overlap():
    """
    Explain why pretrain has overlapping val/test sets
    """
    print("\nWhy does PRETRAIN have overlapping val/test sets?")
    print("=" * 50)
    print("""
    In SELF-SUPERVISED PRETRAINING:
    
    1. GOAL: Learn general time series representations
       - Model learns to predict masked/future values
       - No specific downstream task yet
       
    2. VAL/TEST OVERLAP is acceptable because:
       - You're not doing task-specific evaluation
       - Focus is on representation learning quality
       - Real evaluation happens during fine-tuning
       
    3. MORE TRAINING DATA (16 vs 12 months):
       - Self-supervised learning benefits from more data
       - Better representations with longer sequences
       - Can afford to use more data for pretraining
    
    In STANDARD SUPERVISED LEARNING:
    
    1. GOAL: Optimize for specific forecasting task
       - Model trained end-to-end for prediction
       - Need proper train/val/test separation
       
    2. STRICT SEPARATION required:
       - Prevent data leakage
       - Honest evaluation of generalization
       - Standard ML best practices
    """)

if __name__ == "__main__":
    # Analyze with default sequence length
    splits = analyze_ett_splits(seq_len=96)
    
    # Explain the rationale
    why_pretrain_overlap()
    
    print("\nSUMMARY:")
    print("• pretrain=True: More training data, overlapping val/test (for self-supervised pretraining)")
    print("• pretrain=False: Standard splits with proper separation (for supervised learning)")
    print("• Your ByteLatent training likely uses pretrain=True for representation learning")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def analyze_dataloader_iteration():
    """
    Analyze how the ETT dataloader iterates through channels and time
    """
    
    # Example parameters
    seq_len = 96
    pred_len = 1
    total_time_points = 1000
    n_channels = 7
    
    # Calculate dataset parameters (from ETT code)
    tot_len = total_time_points - seq_len - pred_len + 1  # Valid starting positions per channel
    dataset_len = tot_len * n_channels  # Total dataset length
    
    print("ETT DataLoader Iteration Analysis")
    print("=" * 40)
    print(f"Sequence length: {seq_len}")
    print(f"Total time points: {total_time_points}")
    print(f"Number of channels: {n_channels}")
    print(f"Valid positions per channel: {tot_len}")
    print(f"Total dataset length: {dataset_len}")
    print()
    
    # Show how index mapping works
    print("Index Mapping (from __getitem__):")
    print("-" * 35)
    
    sample_indices = [0, 1, 2, tot_len-1, tot_len, tot_len+1, dataset_len-1]
    
    for idx in sample_indices:
        feat_id = idx // tot_len  # Which channel
        s_begin = idx % tot_len   # Starting time position within that channel
        s_end = s_begin + seq_len
        
        print(f"Index {idx:4d}: Channel {feat_id}, Time [{s_begin:3d}:{s_end:3d}]")
    
    print()
    
    # Detailed channel-by-channel breakdown
    print("Channel-by-Channel Breakdown:")
    print("-" * 30)
    
    for channel in range(n_channels):
        start_idx = channel * tot_len
        end_idx = (channel + 1) * tot_len - 1
        
        # First and last sequence for this channel
        first_time_start = 0
        first_time_end = seq_len
        last_time_start = tot_len - 1
        last_time_end = last_time_start + seq_len
        
        print(f"Channel {channel}:")
        print(f"  Dataset indices: [{start_idx:4d} : {end_idx:4d}]")
        print(f"  First sequence:  Time [{first_time_start:3d} : {first_time_end:3d}]")
        print(f"  Last sequence:   Time [{last_time_start:3d} : {last_time_end:3d}]")
        print()
    
    return tot_len, dataset_len

def visualize_channel_iteration():
    """
    Visualize how sequences are sampled from each channel
    """
    seq_len = 20  # Smaller for visualization
    total_time_points = 50
    n_channels = 3  # Fewer channels for clarity
    
    tot_len = total_time_points - seq_len - 1 + 1  # 30 valid positions
    
    fig, axes = plt.subplots(n_channels, 1, figsize=(12, 8))
    
    colors = ['blue', 'orange', 'green']
    
    for channel in range(n_channels):
        ax = axes[channel]
        
        # Draw the full time series for this channel
        time_points = np.arange(total_time_points)
        channel_data = np.sin(2 * np.pi * time_points / 10) + channel * 2
        ax.plot(time_points, channel_data, color='lightgray', alpha=0.5, linewidth=1)
        
        # Show first few sequences from this channel
        for seq_idx in range(0, min(5, tot_len), max(1, tot_len//10)):
            start_time = seq_idx
            end_time = start_time + seq_len
            
            # Highlight the sequence
            seq_times = np.arange(start_time, end_time)
            seq_values = channel_data[start_time:end_time]
            
            dataset_idx = channel * tot_len + seq_idx
            
            ax.plot(seq_times, seq_values, color=colors[channel], linewidth=3, alpha=0.7,
                   label=f'Dataset idx {dataset_idx}' if seq_idx < 2 else '')
            
            # Mark start and end
            ax.scatter([start_time, end_time-1], [seq_values[0], seq_values[-1]], 
                      color=colors[channel], s=50, zorder=5)
        
        ax.set_title(f'Channel {channel} - Sequences never cross channel boundaries')
        ax.set_xlabel('Time Points')
        ax.set_ylabel('Values')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.suptitle('ETT DataLoader: Each sequence stays within ONE channel', y=1.02)
    plt.show()

def demonstrate_no_mixing():
    """
    Demonstrate that channels never mix
    """
    print("PROOF: No Channel Mixing")
    print("=" * 25)
    print("""
    From the __getitem__ method:
    
    def __getitem__(self, index):
        feat_id = index // self.tot_len     # Which channel (0, 1, 2, ..., 6)
        s_begin = index % self.tot_len      # Time start within that channel
        
        s_end = s_begin + self.seq_len      # Time end within SAME channel
        
        # Key: Only extract from ONE channel
        seq_x = self.data_x[s_begin:s_end, feat_id:feat_id + 1]
                           ^^^^^^^^^^^^^^^  ^^^^^^^^^^^^^^^^^
                           Time slice       Single channel only
    
    IMPORTANT BOUNDARIES:
    """)
    
    # Example with real numbers
    seq_len = 96
    tot_len = 1000  # Example
    n_channels = 7
    
    for channel in range(min(3, n_channels)):  # Show first 3 channels
        channel_start_idx = channel * tot_len
        channel_end_idx = (channel + 1) * tot_len - 1
        
        # Last valid sequence for this channel
        last_seq_start_time = tot_len - 1  # 999
        last_seq_end_time = last_seq_start_time + seq_len  # 1095
        
        print(f"Channel {channel}:")
        print(f"  Dataset indices: [{channel_start_idx} : {channel_end_idx}]")
        print(f"  Last sequence: Time [{last_seq_start_time} : {last_seq_end_time}] (within channel {channel})")
        print(f"  Max time index: {tot_len + seq_len - 2} (still within channel {channel} data)")
        print()
    
    print("CONCLUSION:")
    print("• Each sequence is ENTIRELY from one channel")
    print("• Time boundaries are respected within each channel") 
    print("• NO mixing between channels occurs")
    print("• Each channel is treated as independent time series")

if __name__ == "__main__":
    # Analyze the iteration pattern
    tot_len, dataset_len = analyze_dataloader_iteration()
    
    print("\n" + "="*50)
    
    # Demonstrate no mixing
    demonstrate_no_mixing()
    
    print("\n" + "="*50)
    
    # Visualize the pattern
    visualize_channel_iteration()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

def analyze_batch_channel_distribution(batch_size=64, seq_len=96, total_time_points=10000, n_channels=7):
    """
    Analyze how channels are distributed across batches in ETT dataloader
    """
    
    # Calculate ETT dataset parameters
    tot_len = total_time_points - seq_len - 1 + 1  # Valid starting positions per channel
    dataset_len = tot_len * n_channels  # Total dataset length
    
    print("Batch-Channel Distribution Analysis")
    print("=" * 40)
    print(f"Batch size: {batch_size}")
    print(f"Sequence length: {seq_len}")
    print(f"Total time points: {total_time_points}")
    print(f"Number of channels: {n_channels}")
    print(f"Valid positions per channel (tot_len): {tot_len}")
    print(f"Total dataset length: {dataset_len}")
    print(f"Number of batches: {dataset_len // batch_size}")
    print()
    
    # Analyze first few batches in detail
    print("DETAILED BATCH ANALYSIS:")
    print("-" * 25)
    
    for batch_idx in range(min(10, dataset_len // batch_size)):
        batch_start_idx = batch_idx * batch_size
        batch_end_idx = batch_start_idx + batch_size
        
        channels_in_batch = []
        time_ranges = []
        
        print(f"Batch {batch_idx}: Dataset indices [{batch_start_idx}:{batch_end_idx-1}]")
        
        for sample_idx in range(batch_start_idx, min(batch_end_idx, dataset_len)):
            # ETT's index mapping
            feat_id = sample_idx // tot_len  # Which channel
            s_begin = sample_idx % tot_len   # Time position within channel
            s_end = s_begin + seq_len
            
            channels_in_batch.append(feat_id)
            time_ranges.append((s_begin, s_end))
        
        # Count channels in this batch
        channel_counts = Counter(channels_in_batch)
        
        print(f"  Channels present: {sorted(channel_counts.keys())}")
        print(f"  Channel distribution: {dict(channel_counts)}")
        
        # Show time ranges for first few samples
        print(f"  Sample details (first 5):")
        for i, (channel, (start, end)) in enumerate(zip(channels_in_batch[:5], time_ranges[:5])):
            dataset_idx = batch_start_idx + i
            print(f"    Sample {i}: Dataset[{dataset_idx}] = Channel {channel}, Time [{start}:{end}]")
        
        if len(channels_in_batch) > 5:
            print(f"    ... and {len(channels_in_batch) - 5} more samples")
        print()
    
    return tot_len, dataset_len

def calculate_channel_transition_points(batch_size=64, seq_len=96, total_time_points=10000, n_channels=7):
    """
    Calculate exactly where channel transitions occur in batches
    """
    
    tot_len = total_time_points - seq_len - 1 + 1
    
    print("CHANNEL TRANSITION ANALYSIS:")
    print("-" * 32)
    
    # Find transition points
    transition_batches = []
    
    for channel in range(n_channels - 1):  # 0 to 5 (transition from channel i to i+1)
        # Last index of current channel
        last_idx_current = (channel + 1) * tot_len - 1
        # First index of next channel  
        first_idx_next = (channel + 1) * tot_len
        
        # Which batches do these fall into?
        batch_last_current = last_idx_current // batch_size
        batch_first_next = first_idx_next // batch_size
        
        print(f"Channel {channel} → Channel {channel + 1}:")
        print(f"  Last index of Channel {channel}: {last_idx_current} (Batch {batch_last_current})")
        print(f"  First index of Channel {channel + 1}: {first_idx_next} (Batch {batch_first_next})")
        
        if batch_last_current == batch_first_next:
            print(f"  ⚠️  MIXED BATCH: Batch {batch_last_current} contains BOTH channels {channel} and {channel + 1}")
            
            # Calculate exact positions in the mixed batch
            pos_in_batch_last = last_idx_current % batch_size
            pos_in_batch_first = first_idx_next % batch_size
            
            print(f"     Channel {channel} samples: positions 0 to {pos_in_batch_last}")
            print(f"     Channel {channel + 1} samples: positions {pos_in_batch_first} to {batch_size - 1}")
            
            transition_batches.append({
                'batch_idx': batch_last_current,
                'from_channel': channel,
                'to_channel': channel + 1,
                'split_position': pos_in_batch_first
            })
        else:
            print(f"  ✅ CLEAN: Channels {channel} and {channel + 1} in separate batches")
        print()
    
    return transition_batches

def visualize_batch_channel_distribution(batch_size=64, seq_len=96, total_time_points=1000, n_channels=7):
    """
    Visualize how channels are distributed across batches
    """
    
    tot_len = total_time_points - seq_len - 1 + 1
    dataset_len = tot_len * n_channels
    n_batches = min(20, dataset_len // batch_size)  # Show first 20 batches
    
    # Create batch-channel matrix
    batch_channel_matrix = np.zeros((n_batches, n_channels))
    
    for batch_idx in range(n_batches):
        batch_start_idx = batch_idx * batch_size
        batch_end_idx = min(batch_start_idx + batch_size, dataset_len)
        
        for sample_idx in range(batch_start_idx, batch_end_idx):
            feat_id = sample_idx // tot_len
            if feat_id < n_channels:
                batch_channel_matrix[batch_idx, feat_id] += 1
    
    # Plot heatmap
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    
    # Heatmap of channel counts per batch
    im1 = ax1.imshow(batch_channel_matrix.T, cmap='YlOrRd', aspect='auto')
    ax1.set_xlabel('Batch Index')
    ax1.set_ylabel('Channel ID')
    ax1.set_title(f'Channel Distribution Across Batches (Batch Size = {batch_size})')
    ax1.set_yticks(range(n_channels))
    ax1.set_yticklabels([f'Ch {i}' for i in range(n_channels)])
    
    # Add text annotations
    for i in range(n_batches):
        for j in range(n_channels):
            if batch_channel_matrix[i, j] > 0:
                text = ax1.text(i, j, f'{int(batch_channel_matrix[i, j])}',
                               ha="center", va="center", color="black", fontsize=8)
    
    plt.colorbar(im1, ax=ax1, label='Number of samples')
    
    # Line plot showing channel transitions
    channel_ratios = batch_channel_matrix / batch_size * 100  # Convert to percentages
    
    for channel in range(n_channels):
        ax2.plot(range(n_batches), channel_ratios[:, channel], 
                marker='o', label=f'Channel {channel}', linewidth=2)
    
    ax2.set_xlabel('Batch Index')
    ax2.set_ylabel('Percentage of Batch (%)')
    ax2.set_title('Channel Percentage Distribution Across Batches')
    ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(0, 105)
    
    plt.tight_layout()
    plt.show()
    
    return batch_channel_matrix

def practical_implications(batch_size=64, seq_len=96, total_time_points=10000, n_channels=7):
    """
    Explain practical implications for training and evaluation
    """
    
    tot_len = total_time_points - seq_len - 1 + 1
    
    print("PRACTICAL IMPLICATIONS:")
    print("=" * 23)
    
    print("1. TRAINING IMPLICATIONS:")
    print("   • Each batch may contain MIXED channels")
    print("   • Model sees different channels within same batch")
    print("   • Gradient updates aggregate across multiple channels")
    print("   • This is actually GOOD for learning diverse patterns")
    print()
    
    print("2. EVALUATION IMPLICATIONS:")
    print("   • Need to track which samples belong to which channel")
    print("   • Can't assume batch = single channel")
    print("   • Must use feat_id calculation for channel identification")
    print()
    
    print("3. BATCH SIZE EFFECTS:")
    smaller_batch = 32
    larger_batch = 128
    
    # Calculate transitions for different batch sizes
    for bs in [smaller_batch, batch_size, larger_batch]:
        mixed_batches = 0
        for channel in range(n_channels - 1):
            last_idx = (channel + 1) * tot_len - 1
            first_idx = (channel + 1) * tot_len
            if last_idx // bs == first_idx // bs:
                mixed_batches += 1
        
        print(f"   Batch size {bs}: {mixed_batches}/{n_channels-1} transitions create mixed batches")
    
    print()
    print("4. RECOMMENDATIONS:")
    print("   ✅ Use the provided evaluation code to handle mixed batches")
    print("   ✅ Track channel_id for each sample during inference")
    print("   ✅ Consider batch_size impact on channel mixing")
    print("   ⚠️  Don't assume batch contains only one channel")

if __name__ == "__main__":
    # Example with your parameters
    batch_size = 64
    seq_len = 96
    total_time_points = 10000  # Example ETT dataset size
    n_channels = 7
    
    print("ANALYSIS FOR YOUR PARAMETERS:")
    print("=" * 35)
    
    # Detailed analysis
    tot_len, dataset_len = analyze_batch_channel_distribution(batch_size, seq_len, total_time_points, n_channels)
    
    print("\n" + "="*50)
    
    # Transition analysis
    transitions = calculate_channel_transition_points(batch_size, seq_len, total_time_points, n_channels)
    
    print("\n" + "="*50)
    
    # Practical implications
    practical_implications(batch_size, seq_len, total_time_points, n_channels)
    
    print("\n" + "="*50)
    
    # Visualization
    print("GENERATING VISUALIZATION...")
    batch_channel_matrix = visualize_batch_channel_distribution(batch_size, seq_len, 2000, n_channels)  # Smaller dataset for vis

## Model

In [5]:
model_args = ByteLatentTransformerArgs(
    seed=42,
    vocab_size=4096,                       # Small byte-level vocab
    max_length=seq_len,                        # Max full sequence length
    max_seqlen=seq_len,
    max_encoder_seq_length=seq_len,
    local_attention_window_len=seq_len,        # Local window, 128 is sufficient for small models

    dim_global=128,                        # Lower than default 512
    dim_local_encoder=64,
    dim_local_decoder=64,

    n_layers_global=2,
    n_layers_local_encoder=2,
    n_layers_local_decoder=2,

    n_heads_global=2,                      # Reduce heads
    n_heads_local_encoder=2,
    n_heads_local_decoder=2,

    patch_size=1,
    patch_in_forward=False,                # Patch in forward pass
    patching_batch_size=256,
    patching_device="cuda",               # Use CPU for patching in small model
    patching_mode="entropy",
    patching_threshold=3.0,
    max_patch_length=16,
    monotonicity=True,            # Monotonic patching
    pad_to_max_length=True,

    cross_attn_encoder=True,
    cross_attn_decoder=True,
    cross_attn_k=2,
    cross_attn_nheads=2,
    cross_attn_all_layers_encoder=True,
    cross_attn_all_layers_decoder=True,
    cross_attn_use_flex_attention=False,
    cross_attn_init_by_pooling=True,

    encoder_hash_byte_group_size=[4,5,6],   # Fewer hash sizes
    encoder_hash_byte_group_vocab=2**12,
    encoder_hash_byte_group_nb_functions=1,
    encoder_enable_byte_ngrams=False,

    non_linearity="swiglu",
    use_rope=True,
    attn_impl="sdpa",                      # Efficient PyTorch attention
    attn_bias_type="causal",

    dropout=0.0,
    layer_ckpt="none",                     # No checkpointing in small model
    init_use_gaussian=True,
    init_use_depth="current",
    alpha_depth="disabled",
    log_patch_lengths=True,

    downsampling_by_pooling="max",         # Efficient downsampling
    use_local_encoder_transformer=True,
    share_encoder_decoder_emb=True         # Save memory if possible
)


In [6]:
# model_args = ByteLatentTransformerArgs(
#     seed=42,
#     vocab_size=4096,                       # Small byte-level vocab
#     max_length=seq_len,                        # Max full sequence length
#     max_seqlen=seq_len,
#     max_encoder_seq_length=seq_len,
#     local_attention_window_len=seq_len,        # Local window, 128 is sufficient for small models
#     dim = 128,                            # Lower than default 512
#     # dim_global=512,                        # Lower than default 512
#     dim_local_encoder=128,
#     dim_local_decoder=128,

#     n_layers_global=3,
#     n_layers_local_encoder=2,
#     n_layers_local_decoder=2,

#     n_heads_global=2,                      # Reduce heads
#     n_heads_local_encoder=2,
#     n_heads_local_decoder=2,

#     patch_size=8,
#     patch_in_forward=False,                # Patch in forward pass
#     patching_batch_size=256,
#     patching_device="cuda",               # Use CPU for patching in small model
#     patching_mode="static",
#     patching_threshold=3.0,
#     max_patch_length=16,
#     monotonicity=True,            # Monotonic patching
#     pad_to_max_length=True,

#     cross_attn_encoder=True,
#     cross_attn_decoder=True,
#     cross_attn_k=8,
#     cross_attn_nheads=2,
#     cross_attn_all_layers_encoder=True,
#     cross_attn_all_layers_decoder=True,
#     cross_attn_use_flex_attention=False,
#     cross_attn_init_by_pooling=True,

#     encoder_hash_byte_group_size=[4,5,6,7,8],   # Fewer hash sizes
#     encoder_hash_byte_group_vocab=2**5,
#     encoder_hash_byte_group_nb_functions=1,
#     encoder_enable_byte_ngrams=False,

#     non_linearity="swiglu",
#     use_rope=True,
#     attn_impl="sdpa",                      # Efficient PyTorch attention
#     attn_bias_type="causal",

#     dropout=0.0,
#     layer_ckpt="none",                     # No checkpointing in small model
#     init_use_gaussian=True,
#     init_use_depth="current",
#     alpha_depth="disabled",
#     log_patch_lengths=True,

#     downsampling_by_pooling="max",         # Efficient downsampling
#     use_local_encoder_transformer=True,
#     # share_encoder_decoder_emb=True         # Save memory if possible
# )


In [7]:
model = ByteLatentTransformer(model_args)
model = model.to(device)
if compile:
    model = torch.compile(model)

# n of params in model in millions
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of parameters in model: {count_parameters(model) / 1e6:.2f}M")

Number of parameters in model: 2.19M


In [24]:
def create_static_patch_lengths(batch_size, seq_len, patch_length=8):
    if seq_len == 336:
        n_patches = 43
    elif seq_len == 96:
        n_patches = 13
    elif seq_len == 512:
        n_patches = 65
    elif seq_len == 256:
        n_patches = 33
    elif seq_len == 192:
        n_patches = 13
    elif seq_len == 720:
        n_patches = 91
    else:
        raise ValueError(f"Unsupported seq_len: {seq_len}")

    l = torch.full((batch_size,n_patches), patch_length).to('cuda')
    l[:,0] = 1
    l[:,1] = patch_length - 1
    patch_lengths = l
    return patch_lengths
patch_lengths = create_static_patch_lengths(batch_size=batch_size, seq_len=seq_len)
# patch_lengths = torch.ones((batch_size, seq_len), dtype=torch.int32, device='cuda') * 1
patch_lengths.shape, patch_lengths[0].sum().item()

(torch.Size([64, 13]), 96)

In [25]:
optimizer = optim.AdamW(
        model.parameters(), 
        lr=5e-4, 
        weight_decay=0.01,
        betas=(0.9, 0.95)  # Use better beta values from first code
    )
optimizer.zero_grad(set_to_none=True)

In [26]:
torch.manual_seed(model_args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(model_args.seed)
torch.set_float32_matmul_precision('high')
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16'
scaler = torch.amp.GradScaler(enabled=(dtype == 'float16'))
print(f"Using precision: {dtype}")

Using precision: bfloat16


In [27]:
model

ByteLatentTransformer(
  (local_encoder): LocalEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerBlock(
        (attention): Attention(
          (wq): Linear(in_features=64, out_features=64, bias=False)
          (wk): Linear(in_features=64, out_features=64, bias=False)
          (wv): Linear(in_features=64, out_features=64, bias=False)
          (wo): Linear(in_features=64, out_features=64, bias=False)
        )
        (feed_forward): FeedForward(
          (w1): Linear(in_features=64, out_features=256, bias=False)
          (w3): Linear(in_features=64, out_features=256, bias=False)
          (w2): Linear(in_features=256, out_features=64, bias=False)
        )
        (attention_norm): RMSNorm((64,), eps=1e-05, elementwise_affine=True)
        (ffn_norm): RMSNorm((64,), eps=1e-05, elementwise_affine=True)
      )
    )
    (rope): RotaryEmbedding()
    (patch_embedding_projection): Linear(in_features=64, out_features=128, bias=False)
    (tok_embeddings): Embedding(4096

## Tokenizer

In [28]:
# Create a new config with prediction_length=1
train_tokenizer_config = ChronosConfig(
    tokenizer_class='MeanScaleUniformBins',
    tokenizer_kwargs={'low_limit': -15.0, 'high_limit': 15.0},
    context_length=seq_len,
    prediction_length=seq_len,   
    n_tokens=vocab_size,
    n_special_tokens=4,
    pad_token_id=-1,
    eos_token_id=0,
    use_eos_token=False,
    model_type='causal',
    num_samples=1,
    temperature=1.0,
    top_k=50,
    top_p=1.0
)

# Create a new tokenizer with the updated config
tokenizer = MeanScaleUniformBins(-15, 15, train_tokenizer_config)

## Train Loop

In [29]:
num_batches = len(train_loader)
total_steps = epochs * num_batches
min_lr = learning_rate * min_lr_factor
best_val_loss = float('inf')

# Training loop with validation
for epoch in range(epochs):
    # Training phase
    model.train()
    t1 = time.time()
    epoch_loss = 0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1} Training", position=0, leave=True)
    
    for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in progress_bar:

        iteration = epoch * num_batches + i
        x = batch_x.float().squeeze(-1)
        y = batch_y.float().squeeze(-1)
        
        # Get learning rate
        lr = get_lr(
            iteration,
            total_steps,
            warmup_steps,
            learning_rate,
            min_lr,
            decay_lr
        )
        
        # Update learning rate
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
            
        total_loss = 0
        optimizer.zero_grad(set_to_none=True)
        
        # Gradient accumulation loop
        for micro_step in range(grad_accumulation_steps):
            token_ids, attention_mask, tokenizer_state = tokenizer.context_input_transform(x)
            target_token_ids, target_attention_mask = tokenizer.label_input_transform(y, tokenizer_state)
            
            # Forward pass
            logits = model(token_ids.to(device), patch_lengths)
            predicted_tokens = torch.argmax(logits, dim=-1)
            valid_mask = (target_token_ids != PAD_ID)
            pred_values = predicted_tokens.float().cuda()
            target_values = target_token_ids.float().cuda()
            
            # Calculate loss
            # NEW:
            valid_mask = (target_token_ids != PAD_ID)
            loss, ce_loss, mse_loss = improved_loss_with_gradients(
                logits, target_token_ids.cuda(), valid_mask
            )
            loss = ce_loss / grad_accumulation_steps
            
            # Backward pass
            scaler.scale(loss).backward()
                
            total_loss += loss.item() * grad_accumulation_steps
        
        # Gradient clipping
        if clip_grad > 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)
            
        # Update weights with scaler
        scaler.step(optimizer)
        scaler.update()
        
        # Update training metrics
        epoch_loss += total_loss
        avg_epoch_loss = epoch_loss / (i + 1)
        
        # Update progress bar with current metrics
        progress_bar.set_postfix({
            'loss': f"{total_loss:.4f}",
            'avg_loss': f"{avg_epoch_loss:.4f}",
            'lr': f"{lr:.6f}"
        })
    
    # Calculate training time and average loss
    train_time = time.time() - t1
    train_avg_loss = epoch_loss / len(train_loader)
    
    # Validation phase
    t1 = time.time()
    model.eval()
    val_loss = validate(model, validate_loader, tokenizer, patch_lengths, device, desc=f"Epoch {epoch+1} Validation")
    val_time = time.time() - t1
    
    # Print epoch results
    print(f"\nEpoch {epoch+1}/{epochs} Results:")
    print(f"Training Loss: {train_avg_loss:.4f} (Time: {train_time:.2f}s)")
    print(f"Validation Loss (Avg. MSE): {val_loss:.4f},  (Time: {val_time:.2f}s)")
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scaler': scaler.state_dict() if scaler else None,
            'val_loss': val_loss
        }, os.path.join(output_dir, f'best_model_{features}_{dataset_name}_{seq_len}.pth'))
        print(f"New best model saved with validation loss: {best_val_loss:.4f}")
    
    # Save checkpoint
    if save_every > 0 and (epoch + 1) % save_every == 0:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scaler': scaler.state_dict() if scaler else None,
            'val_loss': val_loss
        }, os.path.join(output_dir, f'checkpoint_{seq_len}_epoch_{epoch+1}.pth'))
        print(f"Checkpoint saved at epoch {epoch+1}")


Epoch 1 Validation: 100%|██████████| 45/45 [00:01<00:00, 39.31it/s, loss=4.3921, avg_loss=4.3431]



Epoch 1/50 Results:
Training Loss: 4.0276 (Time: 4.90s)
Validation Loss (Avg. MSE): 0.1785,  (Time: 1.20s)
New best model saved with validation loss: 0.1785


Epoch 2 Validation: 100%|██████████| 45/45 [00:01<00:00, 32.45it/s, loss=4.2238, avg_loss=4.2963]



Epoch 2/50 Results:
Training Loss: 3.7244 (Time: 4.85s)
Validation Loss (Avg. MSE): 0.1685,  (Time: 1.43s)
New best model saved with validation loss: 0.1685


Epoch 3 Validation: 100%|██████████| 45/45 [00:01<00:00, 34.80it/s, loss=4.3384, avg_loss=4.2312]


...


Epoch 14/50 Results:
Training Loss: 3.0640 (Time: 4.74s)
Validation Loss (Avg. MSE): 0.1416,  (Time: 1.30s)



Epoch 15 Validation: 100%|██████████| 45/45 [00:01<00:00, 37.04it/s, loss=4.4085, avg_loss=4.3239]



Epoch 15/50 Results:
Training Loss: 3.0288 (Time: 4.78s)
Validation Loss (Avg. MSE): 0.1447,  (Time: 1.26s)
Checkpoint saved at epoch 15


Epoch 16 Training:  51%|█████     | 68/133 [00:02<00:02, 26.91it/s, loss=3.0848, avg_loss=2.9817, lr=0.000401]


KeyboardInterrupt: 

## Load Saved Model

In [30]:
# Load and evaluate best model
print("\nEvaluating best model on test set...")
checkpoint = torch.load(os.path.join(output_dir, f"best_model_{features}_{dataset_name}_{seq_len}.pth"))
model.load_state_dict(checkpoint['model_state_dict'])
print(f"Saved Val loss: {checkpoint['val_loss'].item()}")


Evaluating best model on test set...
Saved Val loss: 0.13998858630657196


## Evaluation

### Input 92 >> Predict 96

In [31]:
model.eval()
input_len = 96
pred_len = 96
eval_batch_size = batch_size

max_patch_size = 8  # Maximum tokens per patch
max_new_tokens = pred_len 

_, num_patches = patch_lengths.shape

# Create a new config with prediction_length=1
test_tokenizer_config = ChronosConfig(
    tokenizer_class='MeanScaleUniformBins',
    tokenizer_kwargs={'low_limit': -15.0, 'high_limit': 15.0},
    context_length=input_len,
    prediction_length=pred_len,   
    n_tokens=vocab_size,
    n_special_tokens=4,
    pad_token_id=-1,
    eos_token_id=0,
    use_eos_token=False,
    model_type='causal',
    num_samples=1,
    temperature=1.0,
    top_k=50,
    top_p=1.0
)

# Create a new tokenizer with the updated config
tokenizer = MeanScaleUniformBins(-15, 15, test_tokenizer_config)

In [32]:
config = {
    'task_name': 'pretrain_long_term_forecast',
    'dataset': dataset_name,
    'data' : dataset_name,
    'embed' : 'timeF',
    'root_path': 'dataset/ETT-small/',
    'data_path': f'{dataset_name}.csv',
    'features': features,
    'seq_len': input_len,
    'label_len': 0,
    'pred_len': pred_len
}

args = argparse.Namespace(
    batch_size=eval_batch_size,
    freq='h' if 'h' in dataset_name else 'm',
    num_workers=2,
    subsample_pct=None,
    fix_seed=42,
    target='OT',
    shuffle=False
)

test_dataset, test_loader = data_provider(args, config, flag='test')
for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):
    # Move the batch to the same device as the model
    batch_x = batch_x.to(device)
    batch_y = batch_y.to(device)
    x = batch_x.float().squeeze(-1)
    y = batch_y.float().squeeze(-1)
    token_ids, attention_mask, tokenizer_state = tokenizer.context_input_transform(x.to('cpu'))
    patch_lengths = patch_lengths[0].unsqueeze(0).to(device)
    # tokenizer_state = tokenizer_state[0].unsqueeze(0).to(device)
    target_token_ids, target_attention_mask = tokenizer.label_input_transform(y.to('cpu'), tokenizer_state.to('cpu'))
    break

# patch_lengths = create_static_patch_lengths(batch_size=eval_batch_size, seq_len=input_len)
assert patch_lengths[0].sum().item() == token_ids.shape[1]

for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):
    # Move the batch to the same device as the model
    if i == 0:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        x = batch_x.float().squeeze(-1)
        y = batch_y.float().squeeze(-1)
    break


x.shape, y.shape

test 2785


(torch.Size([64, 96]), torch.Size([64, 96]))

In [33]:
patch_lengths

tensor([[1, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]], device='cuda:0')

In [34]:
top_k = 2  # Top-k filtering
temperature = 1.0 # Temperature for sampling
B, L = token_ids.shape
# Initialize lists to store metrics across batches
all_mse = []
all_mae = []

for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):
    # Move the batch to the same device as the model
    try:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        x = batch_x.float().squeeze(-1)
        y = batch_y.float().squeeze(-1)
        
        # Tokenize input
        token_ids_start, attention_mask, tokenizer_state = tokenizer.context_input_transform(x.to('cpu'))
        target_token_ids, target_attention_mask = tokenizer.label_input_transform(y.to('cpu'), tokenizer_state.to('cpu'))
        
        with torch.no_grad():
            token_ids = token_ids_start
            forecast = torch.zeros((B, max_new_tokens), dtype=torch.long).to(device)
            for _ in range(max_new_tokens):
                # print(_)
                # Get predictions for the next token
                logits = model(token_ids.to(device), patch_lengths)
                #pred_tokens = torch.argmax(logits[:,-1,:], dim=-1)
                #next_token = pred_tokens.unsqueeze(1) 
                next_token_logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)
                
                # # Apply top-k filtering if specified
                if top_k is not None and top_k > 0:
                    v, __ = torch.topk(next_token_logits, min(top_k, next_token_logits.size(-1)))
                    next_token_logits[next_token_logits < v[:, [-1]]] = -float('Inf')
                
                # # Apply softmax to convert to probabilities
                probs = torch.nn.functional.softmax(next_token_logits, dim=-1)
                
                # Sample from the probability distribution
                next_token = torch.multinomial(probs, num_samples=1)
                # print(next_token.shape, forecast.shape, _)
                forecast[:, _] = next_token.squeeze(-1)
                # Update token sequence and patch lengths
                all_tokens = torch.cat([token_ids.to(device), next_token], dim=1)
                look_back = max(pred_len, all_tokens.shape[1])
                token_ids = all_tokens[:, -1*input_len:]  # Keep last 512 tokens

            
            # Extract the forecasted and actual tokens
            actual = target_token_ids

            
            # Convert tokens back to values using inverse transform
            actual_values = tokenizer.output_transform(actual.to('cpu').unsqueeze(1), tokenizer_state.to('cpu'))
            forecast_values = tokenizer.output_transform(forecast.to('cpu').unsqueeze(1), tokenizer_state.to('cpu'))
            
            # Calculate MSE and MAE
            mse = torch.mean((actual_values - forecast_values) ** 2)
            mae = torch.mean(torch.abs(actual_values - forecast_values))
            
            # Store metrics
            all_mse.append(mse.item())
            all_mae.append(mae.item())
            
            # Print for the current batch
            print(f"Batch {i} - MSE: {mse.item():.4f}, MAE: {mae.item():.4f}")
    except Exception as e:
        print(f"Error in batch {i}: {e}")
        continue
    
# Calculate average metrics across all batches
avg_mse = sum(all_mse) / len(all_mse)
avg_mae = sum(all_mae) / len(all_mae)

print(f"\nOverall Results ({seq_len}) ({pred_len}):")
print(f"Average MSE: {avg_mse:.4f}")
print(f"Average MAE: {avg_mae:.4f}")

# Optionally, you can save the metrics to a file
import json
with open(f"forecast_metrics_{dataset_name}_{features}_{seq_len}_{pred_len}.json", "w") as f:
    json.dump({
        "mse_per_batch": all_mse,
        "mae_per_batch": all_mae,
        "average_mse": avg_mse,
        "average_mae": avg_mae
    }, f, indent=4)

Batch 0 - MSE: 0.2983, MAE: 0.4167
Batch 1 - MSE: 0.4019, MAE: 0.5307
Batch 2 - MSE: 0.5645, MAE: 0.6076
Batch 3 - MSE: 0.3845, MAE: 0.4771
Batch 4 - MSE: 0.3125, MAE: 0.4172
Batch 5 - MSE: 0.2308, MAE: 0.3541
Batch 6 - MSE: 0.1333, MAE: 0.2682
Batch 7 - MSE: 0.1472, MAE: 0.2744
Batch 8 - MSE: 0.4938, MAE: 0.5509
Batch 9 - MSE: 1.2583, MAE: 0.9177
Batch 10 - MSE: 0.6931, MAE: 0.5686
Batch 11 - MSE: 0.7439, MAE: 0.6083
Batch 12 - MSE: 0.4563, MAE: 0.4563
Batch 13 - MSE: 0.6926, MAE: 0.5999
Batch 14 - MSE: 0.9255, MAE: 0.6862
Batch 15 - MSE: 1.0834, MAE: 0.7147
Batch 16 - MSE: 1.1563, MAE: 0.7262
Batch 17 - MSE: 1.0209, MAE: 0.6803
Batch 18 - MSE: 1.1145, MAE: 0.7099
Batch 19 - MSE: 0.9185, MAE: 0.6117
Batch 20 - MSE: 1.3136, MAE: 0.7350
Batch 21 - MSE: 1.3730, MAE: 0.7704
Batch 22 - MSE: 1.3461, MAE: 0.7631
Batch 23 - MSE: 1.1259, MAE: 0.6807
Batch 24 - MSE: 0.9917, MAE: 0.5886
Batch 25 - MSE: 0.8000, MAE: 0.6388
Batch 26 - MSE: 1.5002, MAE: 0.9952
Batch 27 - MSE: 1.3612, MAE: 0.7503
Ba

In [35]:
token_ids

tensor([[1887, 1883, 1879,  ..., 1939, 1929, 1929],
        [1883, 1883, 1879,  ..., 2025, 1992, 1982],
        [1879, 1879, 1879,  ..., 1759, 1760, 1783],
        ...,
        [1909, 1909, 1909,  ..., 1894, 1890, 1883],
        [1923, 1923, 1923,  ..., 1952, 1940, 1927],
        [1919, 1919, 1919,  ..., 2097, 2097, 2065]], device='cuda:0')

### Input 96 >> Predict 192

In [ ]:
input_len = 96
pred_len = 192
eval_batch_size = batch_size

max_patch_size = 8  # Maximum tokens per patch
max_new_tokens = pred_len 
B, L = token_ids.shape
_, num_patches = patch_lengths.shape

# Create a new config with prediction_length=1
test_tokenizer_config = ChronosConfig(
    tokenizer_class='MeanScaleUniformBins',
    tokenizer_kwargs={'low_limit': -15.0, 'high_limit': 15.0},
    context_length=input_len,
    prediction_length=pred_len,   
    n_tokens=vocab_size,
    n_special_tokens=4,
    pad_token_id=-1,
    eos_token_id=0,
    use_eos_token=False,
    model_type='causal',
    num_samples=1,
    temperature=1.0,
    top_k=50,
    top_p=1.0
)

# Create a new tokenizer with the updated config
tokenizer = MeanScaleUniformBins(-15, 15, test_tokenizer_config)

In [ ]:
config = {
    'task_name': 'pretrain_long_term_forecast',
    'dataset': dataset_name,
    'data' : dataset_name,
    'embed' : 'timeF',
    'root_path': 'dataset/ETT-small/',
    'data_path': f'{dataset_name}.csv',
    'features': features,
    'seq_len': input_len,
    'label_len': 0,
    'pred_len': pred_len
}

args = argparse.Namespace(
    batch_size=eval_batch_size,
    freq='h' if 'h' in dataset_name else 'm',
    num_workers=2,
    subsample_pct=None,
    fix_seed=42,
    target='OT',
    shuffle=False
)
test_dataset, test_loader = data_provider(args, config, flag='test')
for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):
    # Move the batch to the same device as the model
    batch_x = batch_x.to(device)
    batch_y = batch_y.to(device)
    x = batch_x.float().squeeze(-1)
    y = batch_y.float().squeeze(-1)
    token_ids, attention_mask, tokenizer_state = tokenizer.context_input_transform(x.to('cpu'))
    patch_lengths = patch_lengths[0].unsqueeze(0).to(device)
    # tokenizer_state = tokenizer_state[0].unsqueeze(0).to(device)
    target_token_ids, target_attention_mask = tokenizer.label_input_transform(y.to('cpu'), tokenizer_state.to('cpu'))
    break

patch_lengths = create_static_patch_lengths(batch_size=eval_batch_size, seq_len=input_len)
assert patch_lengths[0].sum().item() == token_ids.shape[1]

for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):
    # Move the batch to the same device as the model
    if i == 0:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        x = batch_x.float().squeeze(-1)
        y = batch_y.float().squeeze(-1)
    break

x.shape, y.shape

In [ ]:
# Initialize lists to store metrics across batches
all_mse = []
all_mae = []

for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):
    # Move the batch to the same device as the model
    try:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        x = batch_x.float().squeeze(-1)
        y = batch_y.float().squeeze(-1)
        
        # Tokenize input
        token_ids_start, attention_mask, tokenizer_state = tokenizer.context_input_transform(x.to('cpu'))
        target_token_ids, target_attention_mask = tokenizer.label_input_transform(y.to('cpu'), tokenizer_state.to('cpu'))
        
        with torch.no_grad():
            token_ids = token_ids_start
            forecast = torch.zeros((B, max_new_tokens), dtype=torch.long).to(device)
            for _ in range(max_new_tokens):
                # print(_)
                # Get predictions for the next token
                logits = model(token_ids.to(device), patch_lengths)
                pred_tokens = torch.argmax(logits[:,-1,:], dim=-1)
                next_token = pred_tokens.unsqueeze(1) 
                # next_token_logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)
                
                # # Apply top-k filtering if specified
                # if top_k is not None and top_k > 0:
                #     v, __ = torch.topk(next_token_logits, min(top_k, next_token_logits.size(-1)))
                #     next_token_logits[next_token_logits < v[:, [-1]]] = -float('Inf')
                
                # # Apply softmax to convert to probabilities
                # probs = torch.nn.functional.softmax(next_token_logits, dim=-1)
                
                # Sample from the probability distribution
                # next_token = torch.multinomial(probs, num_samples=1)
                # print(next_token.shape, forecast.shape, _)
                forecast[:, _] = next_token.squeeze(-1)
                # Update token sequence and patch lengths
                all_tokens = torch.cat([token_ids.to(device), next_token], dim=1)
                look_back = max(pred_len, all_tokens.shape[1])
                token_ids = all_tokens[:, -1*input_len:]  # Keep last 512 tokens

            
            # Extract the forecasted and actual tokens
            actual = target_token_ids

            
            # Convert tokens back to values using inverse transform
            actual_values = tokenizer.output_transform(actual.to('cpu').unsqueeze(1), tokenizer_state.to('cpu'))
            forecast_values = tokenizer.output_transform(forecast.to('cpu').unsqueeze(1), tokenizer_state.to('cpu'))
            
            # Calculate MSE and MAE
            mse = torch.mean((actual_values - forecast_values) ** 2)
            mae = torch.mean(torch.abs(actual_values - forecast_values))
            
            # Store metrics
            all_mse.append(mse.item())
            all_mae.append(mae.item())
            
            # Print for the current batch
            print(f"Batch {i} - MSE: {mse.item():.4f}, MAE: {mae.item():.4f}")
    except Exception as e:
        print(f"Error in batch {i}: {e}")
        continue
    
# Calculate average metrics across all batches
avg_mse = sum(all_mse) / len(all_mse)
avg_mae = sum(all_mae) / len(all_mae)

print(f"\nOverall Results ({seq_len}) ({pred_len}):")
print(f"Average MSE: {avg_mse:.4f}")
print(f"Average MAE: {avg_mae:.4f}")

# Optionally, you can save the metrics to a file
import json
with open(f"forecast_metrics_{dataset_name}_{features}_{input_len}_{pred_len}.json", "w") as f:
    json.dump({
        "mse_per_batch": all_mse,
        "mae_per_batch": all_mae,
        "average_mse": avg_mse,
        "average_mae": avg_mae
    }, f, indent=4)

### Input 96 >> Predict 336

In [ ]:
input_len = 96
pred_len = 336
eval_batch_size = batch_size

max_patch_size = 8  # Maximum tokens per patch
max_new_tokens = pred_len 
B, L = token_ids.shape
_, num_patches = patch_lengths.shape

# Create a new config with prediction_length=1
test_tokenizer_config = ChronosConfig(
    tokenizer_class='MeanScaleUniformBins',
    tokenizer_kwargs={'low_limit': -15.0, 'high_limit': 15.0},
    context_length=input_len,
    prediction_length=pred_len,   
    n_tokens=vocab_size,
    n_special_tokens=4,
    pad_token_id=-1,
    eos_token_id=0,
    use_eos_token=False,
    model_type='causal',
    num_samples=1,
    temperature=1.0,
    top_k=50,
    top_p=1.0
)

# Create a new tokenizer with the updated config
tokenizer = MeanScaleUniformBins(-15, 15, test_tokenizer_config)

In [ ]:
config = {
    'task_name': 'pretrain_long_term_forecast',
    'dataset': dataset_name,
    'data' : dataset_name,
    'embed' : 'timeF',
    'root_path': 'dataset/ETT-small/',
    'data_path': f'{dataset_name}.csv',
    'features': features,
    'seq_len': input_len,
    'label_len': 0,
    'pred_len': pred_len
}

args = argparse.Namespace(
    batch_size=eval_batch_size,
    freq='h' if 'h' in dataset_name else 'm',
    num_workers=2,
    subsample_pct=None,
    fix_seed=42,
    target='OT',
    shuffle=False
)
test_dataset, test_loader = data_provider(args, config, flag='test')
for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):
    # Move the batch to the same device as the model
    batch_x = batch_x.to(device)
    batch_y = batch_y.to(device)
    x = batch_x.float().squeeze(-1)
    y = batch_y.float().squeeze(-1)
    token_ids, attention_mask, tokenizer_state = tokenizer.context_input_transform(x.to('cpu'))
    patch_lengths = patch_lengths[0].unsqueeze(0).to(device)
    # tokenizer_state = tokenizer_state[0].unsqueeze(0).to(device)
    target_token_ids, target_attention_mask = tokenizer.label_input_transform(y.to('cpu'), tokenizer_state.to('cpu'))
    break

patch_lengths = create_static_patch_lengths(batch_size=eval_batch_size, seq_len=input_len)
assert patch_lengths[0].sum().item() == token_ids.shape[1]

for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):
    # Move the batch to the same device as the model
    if i == 0:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        x = batch_x.float().squeeze(-1)
        y = batch_y.float().squeeze(-1)
    break

x.shape, y.shape

In [ ]:
# Initialize lists to store metrics across batches
all_mse = []
all_mae = []

for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):
    # Move the batch to the same device as the model
    try:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        x = batch_x.float().squeeze(-1)
        y = batch_y.float().squeeze(-1)
        
        # Tokenize input
        token_ids_start, attention_mask, tokenizer_state = tokenizer.context_input_transform(x.to('cpu'))
        target_token_ids, target_attention_mask = tokenizer.label_input_transform(y.to('cpu'), tokenizer_state.to('cpu'))
        
        with torch.no_grad():
            token_ids = token_ids_start
            forecast = torch.zeros((B, max_new_tokens), dtype=torch.long).to(device)
            for _ in range(max_new_tokens):
                # print(_)
                # Get predictions for the next token
                logits = model(token_ids.to(device), patch_lengths)
                pred_tokens = torch.argmax(logits[:,-1,:], dim=-1)
                next_token = pred_tokens.unsqueeze(1) 
                # next_token_logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)
                
                # # Apply top-k filtering if specified
                # if top_k is not None and top_k > 0:
                #     v, __ = torch.topk(next_token_logits, min(top_k, next_token_logits.size(-1)))
                #     next_token_logits[next_token_logits < v[:, [-1]]] = -float('Inf')
                
                # # Apply softmax to convert to probabilities
                # probs = torch.nn.functional.softmax(next_token_logits, dim=-1)
                
                # Sample from the probability distribution
                # next_token = torch.multinomial(probs, num_samples=1)
                # print(next_token.shape, forecast.shape, _)
                forecast[:, _] = next_token.squeeze(-1)
                # Update token sequence and patch lengths
                all_tokens = torch.cat([token_ids.to(device), next_token], dim=1)
                look_back = max(pred_len, all_tokens.shape[1])
                token_ids = all_tokens[:, -1*input_len:]  # Keep last 512 tokens

            
            # Extract the forecasted and actual tokens
            actual = target_token_ids

            
            # Convert tokens back to values using inverse transform
            actual_values = tokenizer.output_transform(actual.to('cpu').unsqueeze(1), tokenizer_state.to('cpu'))
            forecast_values = tokenizer.output_transform(forecast.to('cpu').unsqueeze(1), tokenizer_state.to('cpu'))
            
            # Calculate MSE and MAE
            mse = torch.mean((actual_values - forecast_values) ** 2)
            mae = torch.mean(torch.abs(actual_values - forecast_values))
            
            # Store metrics
            all_mse.append(mse.item())
            all_mae.append(mae.item())
            
            # Print for the current batch
            print(f"Batch {i} - MSE: {mse.item():.4f}, MAE: {mae.item():.4f}")
    except Exception as e:
        print(f"Error in batch {i}: {e}")
        continue
    
# Calculate average metrics across all batches
avg_mse = sum(all_mse) / len(all_mse)
avg_mae = sum(all_mae) / len(all_mae)

print(f"\nOverall Results ({seq_len}) ({pred_len}):")
print(f"Average MSE: {avg_mse:.4f}")
print(f"Average MAE: {avg_mae:.4f}")

# Optionally, you can save the metrics to a file
import json
with open(f"forecast_metrics_{dataset_name}_{features}_{input_len}_{pred_len}.json", "w") as f:
    json.dump({
        "mse_per_batch": all_mse,
        "mae_per_batch": all_mae,
        "average_mse": avg_mse,
        "average_mae": avg_mae
    }, f, indent=4)

### Input 96 >> Predict 720

In [ ]:
input_len = 96
pred_len = 720
eval_batch_size = batch_size

max_patch_size = 8  # Maximum tokens per patch
max_new_tokens = pred_len 
B, L = token_ids.shape
_, num_patches = patch_lengths.shape

# Create a new config with prediction_length=1
test_tokenizer_config = ChronosConfig(
    tokenizer_class='MeanScaleUniformBins',
    tokenizer_kwargs={'low_limit': -15.0, 'high_limit': 15.0},
    context_length=input_len,
    prediction_length=pred_len,   
    n_tokens=vocab_size,
    n_special_tokens=4,
    pad_token_id=-1,
    eos_token_id=0,
    use_eos_token=False,
    model_type='causal',
    num_samples=1,
    temperature=1.0,
    top_k=50,
    top_p=1.0
)

# Create a new tokenizer with the updated config
tokenizer = MeanScaleUniformBins(-15, 15, test_tokenizer_config)

In [ ]:
config = {
    'task_name': 'pretrain_long_term_forecast',
    'dataset': dataset_name,
    'data' : dataset_name,
    'embed' : 'timeF',
    'root_path': 'dataset/ETT-small/',
    'data_path': f'{dataset_name}.csv',
    'features': features,
    'seq_len': input_len,
    'label_len': 0,
    'pred_len': pred_len
}

args = argparse.Namespace(
    batch_size=eval_batch_size,
    freq='h' if 'h' in dataset_name else 'm',
    num_workers=2,
    subsample_pct=None,
    fix_seed=42,
    target='OT',
    shuffle=False
)
test_dataset, test_loader = data_provider(args, config, flag='test')

for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):
    # Move the batch to the same device as the model
    batch_x = batch_x.to(device)
    batch_y = batch_y.to(device)
    x = batch_x.float().squeeze(-1)
    y = batch_y.float().squeeze(-1)
    token_ids, attention_mask, tokenizer_state = tokenizer.context_input_transform(x.to('cpu'))
    patch_lengths = patch_lengths[0].unsqueeze(0).to(device)
    # tokenizer_state = tokenizer_state[0].unsqueeze(0).to(device)
    target_token_ids, target_attention_mask = tokenizer.label_input_transform(y.to('cpu'), tokenizer_state.to('cpu'))
    break

patch_lengths = create_static_patch_lengths(batch_size=eval_batch_size, seq_len=input_len)
assert patch_lengths[0].sum().item() == token_ids.shape[1]

for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):
    # Move the batch to the same device as the model
    if i == 0:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        x = batch_x.float().squeeze(-1)
        y = batch_y.float().squeeze(-1)
    break

x.shape, y.shape

In [ ]:
# Initialize lists to store metrics across batches
all_mse = []
all_mae = []

for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):
    # Move the batch to the same device as the model
    try:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        x = batch_x.float().squeeze(-1)
        y = batch_y.float().squeeze(-1)
        
        # Tokenize input
        token_ids_start, attention_mask, tokenizer_state = tokenizer.context_input_transform(x.to('cpu'))
        target_token_ids, target_attention_mask = tokenizer.label_input_transform(y.to('cpu'), tokenizer_state.to('cpu'))
        
        with torch.no_grad():
            token_ids = token_ids_start
            forecast = torch.zeros((B, max_new_tokens), dtype=torch.long).to(device)
            for _ in range(max_new_tokens):
                # print(_)
                # Get predictions for the next token
                logits = model(token_ids.to(device), patch_lengths)
                pred_tokens = torch.argmax(logits[:,-1,:], dim=-1)
                next_token = pred_tokens.unsqueeze(1) 
                # next_token_logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)
                
                # # Apply top-k filtering if specified
                # if top_k is not None and top_k > 0:
                #     v, __ = torch.topk(next_token_logits, min(top_k, next_token_logits.size(-1)))
                #     next_token_logits[next_token_logits < v[:, [-1]]] = -float('Inf')
                
                # # Apply softmax to convert to probabilities
                # probs = torch.nn.functional.softmax(next_token_logits, dim=-1)
                
                # Sample from the probability distribution
                # next_token = torch.multinomial(probs, num_samples=1)
                # print(next_token.shape, forecast.shape, _)
                forecast[:, _] = next_token.squeeze(-1)
                # Update token sequence and patch lengths
                all_tokens = torch.cat([token_ids.to(device), next_token], dim=1)
                look_back = max(pred_len, all_tokens.shape[1])
                token_ids = all_tokens[:, -1*input_len:]  # Keep last 512 tokens

            
            # Extract the forecasted and actual tokens
            actual = target_token_ids

            
            # Convert tokens back to values using inverse transform
            actual_values = tokenizer.output_transform(actual.to('cpu').unsqueeze(1), tokenizer_state.to('cpu'))
            forecast_values = tokenizer.output_transform(forecast.to('cpu').unsqueeze(1), tokenizer_state.to('cpu'))
            
            # Calculate MSE and MAE
            mse = torch.mean((actual_values - forecast_values) ** 2)
            mae = torch.mean(torch.abs(actual_values - forecast_values))
            
            # Store metrics
            all_mse.append(mse.item())
            all_mae.append(mae.item())
            
            # Print for the current batch
            print(f"Batch {i} - MSE: {mse.item():.4f}, MAE: {mae.item():.4f}")
    except Exception as e:
        print(f"Error in batch {i}: {e}")
        continue
    
# Calculate average metrics across all batches
avg_mse = sum(all_mse) / len(all_mse)
avg_mae = sum(all_mae) / len(all_mae)

print(f"\nOverall Results ({seq_len}) ({pred_len}):")
print(f"Average MSE: {avg_mse:.4f}")
print(f"Average MAE: {avg_mae:.4f}")

# Optionally, you can save the metrics to a file
import json
with open(f"forecast_metrics_{dataset_name}_{features}_{input_len}_{pred_len}.json", "w") as f:
    json.dump({
        "mse_per_batch": all_mse,
        "mae_per_batch": all_mae,
        "average_mse": avg_mse,
        "average_mae": avg_mae
    }, f, indent=4)